In [1]:
!pip install substrate-interface

Defaulting to user installation because normal site-packages is not writeable
     |████████████████████████████████| 182 kB 6.3 MB/s            
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     |████████████████████████████████| 2.0 MB 50.1 MB/s            
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     |████████████████████████████████| 243 kB 66.5 MB/s            
     |████████████████████████████████| 119 kB 68.2 MB/s            
     |████████████████████████████████| 75 kB 6.8 MB/s             
     |████████████████████████████████| 856 kB 75.7 MB/s            
     |████████████████████████████████| 481 kB 74.2 MB/s            
  Preparing metadata (setup.py) ... 

In [1]:
import substrateinterface

In [2]:
from substrateinterface import SubstrateInterface

In [3]:
from substrateinterface import SubstrateInterface, Keypair
from substrateinterface.exceptions import SubstrateRequestException

substrate = SubstrateInterface(
    url="ws://35.80.10.26:11946",
    ss58_format=42,
    type_registry_preset='kusama'
)

In [4]:
# Function to make extrinsic calls
def make_call(call_module, call_function, call_params, keypair):
    call = substrate.compose_call(
        call_module=call_module,
        call_function=call_function,
        call_params=call_params
    )

    extrinsic = substrate.create_signed_extrinsic(call=call, keypair=keypair)

    try:
        receipt = substrate.submit_extrinsic(extrinsic, wait_for_inclusion=True)
        print("Extrinsic '{}' sent and included in block '{}'".format(receipt.extrinsic_hash, receipt.block_hash))

    except SubstrateRequestException as e:
        print("Failed to send: {}".format(e))
    return receipt

In [5]:
alice = Keypair.create_from_uri('//Alice')
bob = Keypair.create_from_uri('//Bob')

In [57]:
# Every wallet that creates messages needs a msa
make_call("Msa", "create", {}, alice)

Extrinsic '0x767ab4bafa20545e94e0980cf73ac6ce8b57d442ca46f7308fc5ea479781fc51' sent and included in block '0xcc64497e3f0ab68ebdd522eed1a5568f7a3056e99ffa3a84115c511b2bd190a0'


In [ ]:
# A wallet can create an MSA and delegate it to another wallet, which will submit and pay for transactions on their behave
make_call("Msa", "create", {}, bob)

In [5]:
schema = "subreddit,author,title,selftext,url,is_nsfw"

schema_count = substrate.query(
    module='Schemas',
    storage_function='SchemaCount',
    params=[]
).value

schemaId = -1
for i in range(1, schema_count+1):
    schemaTemp = substrate.query(
        module='Schemas',
        storage_function='Schemas',
        params=[i]
    )
    if schemaTemp == schema:
        schemaId = i
        print(schemaTemp.value)
        break
    
if schemaId == -1:
    make_call("Schemas", "register_schema", {schema}, alice)

print(schemaId)

subreddit,author,title,selftext,url,is_nsfw
1


In [9]:
message = "AskReddit,LazarShockX,What are some good ice breaker questions?,,https://www.reddit.com/r/AskReddit/comments/v5ihpk/what_are_some_good_ice_breaker_questions/,False"

call_params = {
    "schema_id": schemaId,
    "message": message
}
receipt = make_call("Messages", "add", call_params, alice)

Extrinsic '0x9e92cb7b10ed9f683b2f7a3fb3b3b95a0b70c97950be8e61154e4bde12b8767e' sent and included in block '0xb9ab002011ef2db72da23f2f08fbe9570b39292a6120be32127d9763d6fbe96a'


In [10]:
receipt.triggered_events

[<scale_info::16(value={'phase': 'ApplyExtrinsic', 'extrinsic_idx': 2, 'event': {'event_index': '0a08', 'module_id': 'Balances', 'event_id': 'Withdraw', 'attributes': ('5GrwvaEF5zXb26Fz9rcQpDWS57CtERHpNehXCPcNoHGKutQY', 2734463659)}, 'event_index': 10, 'module_id': 'Balances', 'event_id': 'Withdraw', 'attributes': ('5GrwvaEF5zXb26Fz9rcQpDWS57CtERHpNehXCPcNoHGKutQY', 2734463659), 'topics': []})>,
 <scale_info::16(value={'phase': 'ApplyExtrinsic', 'extrinsic_idx': 2, 'event': {'event_index': '0a07', 'module_id': 'Balances', 'event_id': 'Deposit', 'attributes': ('5GrwvaEF5zXb26Fz9rcQpDWS57CtERHpNehXCPcNoHGKutQY', 2487617)}, 'event_index': 10, 'module_id': 'Balances', 'event_id': 'Deposit', 'attributes': ('5GrwvaEF5zXb26Fz9rcQpDWS57CtERHpNehXCPcNoHGKutQY', 2487617), 'topics': []})>,
 <scale_info::16(value={'phase': 'ApplyExtrinsic', 'extrinsic_idx': 2, 'event': {'event_index': '0000', 'module_id': 'System', 'event_id': 'ExtrinsicSuccess', 'attributes': {'weight': 125163000, 'class': 'Norma

In [11]:
receipt.extrinsic

<GenericExtrinsic(value={'extrinsic_hash': '0x9e92cb7b10ed9f683b2f7a3fb3b3b95a0b70c97950be8e61154e4bde12b8767e', 'extrinsic_length': 271, 'address': '5GrwvaEF5zXb26Fz9rcQpDWS57CtERHpNehXCPcNoHGKutQY', 'signature': {'Sr25519': '0xec8bf60cfc44863edaff635cf598a7f695b2243e06cc251be978af5c952ef4104e24597b4e3fa24b050171d38fbf9a6f23d9556f5d071fbbd61ed4f508ca3987'}, 'era': '00', 'nonce': 10, 'tip': 0, 'call': {'call_index': '0x2300', 'call_function': 'add', 'call_module': 'Messages', 'call_args': [{'name': 'schema_id', 'type': 'SchemaId', 'value': 1}, {'name': 'message', 'type': 'Bytes', 'value': 'AskReddit,LazarShockX,What are some good ice breaker questions?,,https://www.reddit.com/r/AskReddit/comments/v5ihpk/what_are_some_good_ice_breaker_questions/,False'}], 'call_hash': '0x03864c06b4716443830a7d7622cb3f82c67a15eb9010b5a8fa3d491e6af5d696'}})>

In [12]:
receipt.extrinsic_idx

2

In [26]:

ferdie = Keypair.create_from_uri('//Ferdie')
receipt = make_call("Messages", "add", call_params, ferdie)

Extrinsic '0x9e55f536ad8044e79742a295a2bea09bfa530f626a7181bc32062da8f0d931b9' sent and included in block '0xf8a47797a09062afbae1dcad96be4152a320cc41200b4a1b7e8b55b040952bc7'


In [35]:
receipt.error_message

{'type': 'Module',
 'name': 'InvalidMessageSourceAccount',
 'docs': ['Invalid Message Source Account']}

In [36]:
alice.sign?

In [42]:
alice.encrypt_message({ "authorized_msa_id": 1, "permission": 0 })

TypeError: encrypt_message() missing 1 required positional argument: 'recipient_public_key'

# Make MSA for new wallet and delegate to bob

In [132]:
!pip install scalecodec

Defaulting to user installation because normal site-packages is not writeable


In [6]:
import scalecodec

In [7]:
from scalecodec import ScaleBytes
from scalecodec.base import RuntimeConfigurationObject

In [8]:
# Create never before used wallet
new_wallet = Keypair.create_from_uri('/NewWallet', crypto_type=substrateinterface.KeypairType.SR25519)

In [53]:
# Give wallet some tokens to it can make MSA
# Must be at least 1 unit
one_unit = 1000000000000
receipt = make_call("Balances", "transfer", {"dest": new_wallet.ss58_address, "value": 10 * one_unit}, alice)
receipt.error_message

Extrinsic '0xbe6844e97cd09cffc88a0bbf737388f129b18e70d4757280ab4e9ac561a389a5' sent and included in block '0x7b5df921a3a39d25bdd92a46400eab3e81672ae4f936c15a28d3c26bd72dcb1b'


In [56]:
receipt = make_call("Msa", "create", {}, new_wallet)
receipt.error_message

{'call': '0x2200', 'era': '00', 'nonce': 11, 'tip': 0, 'spec_version': 1, 'genesis_hash': '0xe2ba7ea2c82e4de81244f86c84d4f3f267103104ce5103535f3f1bd2c834f3b6', 'block_hash': '0xe2ba7ea2c82e4de81244f86c84d4f3f267103104ce5103535f3f1bd2c834f3b6', 'transaction_version': 1, 'asset_id': {'tip': 0, 'asset_id': None}} call CallBytes b'"\x00' 0x2200

{'call': '0x2200', 'era': '00', 'nonce': 11, 'tip': 0, 'spec_version': 1, 'genesis_hash': '0xe2ba7ea2c82e4de81244f86c84d4f3f267103104ce5103535f3f1bd2c834f3b6', 'block_hash': '0xe2ba7ea2c82e4de81244f86c84d4f3f267103104ce5103535f3f1bd2c834f3b6', 'transaction_version': 1, 'asset_id': {'tip': 0, 'asset_id': None}} era scale_info::265 0x00 0x220000

{'call': '0x2200', 'era': '00', 'nonce': 11, 'tip': 0, 'spec_version': 1, 'genesis_hash': '0xe2ba7ea2c82e4de81244f86c84d4f3f267103104ce5103535f3f1bd2c834f3b6', 'block_hash': '0xe2ba7ea2c82e4de81244f86c84d4f3f267103104ce5103535f3f1bd2c834f3b6', 'transaction_version': 1, 'asset_id': {'tip': 0, 'asset_id': None

Extrinsic '0xceb65d0968eac3b93e577049f7242fc0f93fb5759eb670a7f161f62335e283a5' sent and included in block '0xc73931357324ff63708de8521fed64a07c66fe4136f5bf827f3e2ceac8a64991'


In [61]:
receipt.triggered_events

[<scale_info::16(value={'phase': 'ApplyExtrinsic', 'extrinsic_idx': 2, 'event': {'event_index': '0a08', 'module_id': 'Balances', 'event_id': 'Withdraw', 'attributes': ('5Gge15WwoSeyPiGy9FzJRipLFGma3RkaX7G1gez6pbGAGnNC', 1065060100)}, 'event_index': 10, 'module_id': 'Balances', 'event_id': 'Withdraw', 'attributes': ('5Gge15WwoSeyPiGy9FzJRipLFGma3RkaX7G1gez6pbGAGnNC', 1065060100), 'topics': []})>,
 <scale_info::16(value={'phase': 'ApplyExtrinsic', 'extrinsic_idx': 2, 'event': {'event_index': '2200', 'module_id': 'Msa', 'event_id': 'MsaCreated', 'attributes': (3, '5Gge15WwoSeyPiGy9FzJRipLFGma3RkaX7G1gez6pbGAGnNC')}, 'event_index': 34, 'module_id': 'Msa', 'event_id': 'MsaCreated', 'attributes': (3, '5Gge15WwoSeyPiGy9FzJRipLFGma3RkaX7G1gez6pbGAGnNC'), 'topics': []})>,
 <scale_info::16(value={'phase': 'ApplyExtrinsic', 'extrinsic_idx': 2, 'event': {'event_index': '0000', 'module_id': 'System', 'event_id': 'ExtrinsicSuccess', 'attributes': {'weight': 524028000, 'class': 'Normal', 'pays_fee': 

In [105]:
# encode payload with 
payload_raw = { "msa_id": 3, "nonce": 0 }
# add_provider_payload = substrate.encode_scale('str', substrateinterface.utils.ecdsa_helpers.struct.pack("bb", 1, 0))
add_provider_payload = substrate.encode_scale(type_string='scale_info::8', value=payload_raw['msa_id']) + substrate.encode_scale(type_string='scale_info::4', value=payload_raw['nonce'])

In [109]:
"<Bytes>".encode()

b'<Bytes>'

In [111]:
"<Bytes>".encode()+add_provider_payload

TypeError: can't concat ScaleBytes to bytes

In [106]:
signature = new_wallet.sign(add_provider_payload)

In [90]:
signature = new_wallet.sign(blake2b(add_provider_payload.data, digest_size=32).digest())

In [107]:
call_params = {
    "key": new_wallet.ss58_address,
    "proof": {"Sr25519": "0x" + signature.hex()},
    "add_key_payload": payload_raw
}

receipt = make_call("Msa", "add_key_to_msa", call_params, new_wallet)

{'Sr25519': '0x80dd089bfe4d60dc2a63ddccfe3b6b1aa284b0b6d4bc3a46bf59b38f72e70b5b329cc9016658361d91a0484b05a527747d43c17ae9f34000cacbe465b643048a'} Sr25519 scale_info::185 0x80dd089bfe4d60dc2a63ddccfe3b6b1aa284b0b6d4bc3a46bf59b38f72e70b5b329cc9016658361d91a0484b05a527747d43c17ae9f34000cacbe465b643048a 0x80dd089bfe4d60dc2a63ddccfe3b6b1aa284b0b6d4bc3a46bf59b38f72e70b5b329cc9016658361d91a0484b05a527747d43c17ae9f34000cacbe465b643048a

{'msa_id': 3, 'nonce': 0} msa_id scale_info::8 0x0300000000000000 0x0300000000000000

{'msa_id': 3, 'nonce': 0} nonce scale_info::4 0x00000000 0x030000000000000000000000

{'call': '0x2204cc58ca91355d26d9fe826cc0ded719e219f4e798c698c01f407844a5f8682c4f0180dd089bfe4d60dc2a63ddccfe3b6b1aa284b0b6d4bc3a46bf59b38f72e70b5b329cc9016658361d91a0484b05a527747d43c17ae9f34000cacbe465b643048a030000000000000000000000', 'era': '00', 'nonce': 17, 'tip': 0, 'spec_version': 1, 'genesis_hash': '0xe2ba7ea2c82e4de81244f86c84d4f3f267103104ce5103535f3f1bd2c834f3b6', 'block_hash': '0xe

Extrinsic '0x64b5ae209ce0eecce290c47dc5680ad867df4f9e01dac3e024fc34d2eb2cd6a1' sent and included in block '0xfa476495051bf6d3e10de157bbfb53e0ab8481d1185e557c27f20d52b4f4678d'


In [108]:
receipt.error_message

{'type': 'Module',
 'name': 'AddKeySignatureVerificationFailed',
 'docs': ['Cryptographic signature verification failed for adding a key to MSA']}

In [67]:
t = substrate.create_scale_object(type_string='scale_info::8', metadata=substrate.metadata_decoder)

In [69]:
t.encode(1) == substrate.encode_scale(type_string='scale_info::8', value=1)

True

In [93]:
add_provider_payload.to_hex() == '0x030000000000000000'

True

In [96]:
new_wallet.verify(add_provider_payload, "0x" + signature.hex())

True